In [1]:
import pandas as pd
import numpy as np

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import pipeline

In [2]:
from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read in filtered data from Google Drive
#file_path = '/content/drive/MyDrive/filtered_data.csv'
# Read in filtered data locally
file_path = 'data/filtered_data.csv'
df = pd.read_csv(file_path)
df.shape

In [21]:
# Select relevant fields: other_moods, other_lifestyles, notes
text_df = pd.DataFrame(df, columns=['ID', 'userID', 'other_moods', 'other_lifestyles', 'notes'])

users_total = len(text_df['userID'].unique())

In [34]:
# Drop rows where relevant fields are NaN
text_df = text_df.dropna(how='all', subset=['other_moods', 'other_lifestyles', 'notes'])
print(text_df.shape)

users_text = len(text_df['userID'].unique())
print(users_total)
print(users_text)

print("Percentage of users that made text notes: ", round(users_text*100/users_total, 1), "%")

(6240, 5)
1932
1032
Percentage of users that made text notes:  53.4 %


In [11]:
# Ensure fields are strings
text_df["other_moods"] = text_df["other_moods"].astype(str)
text_df["other_lifestyles"] = text_df["other_lifestyles"].astype(str)
text_df["notes"] = text_df["notes"].astype(str)

# Combine text fields, drop individual text fields
text_df["text"] = text_df["other_moods"] + " " + text_df["other_lifestyles"] + " " + text_df["notes"]
text_df = text_df.drop(columns=['other_moods', 'other_lifestyles', 'notes'])

# Add a column for work related label
text_df['label'] = ''
text_df.shape

(6240, 4)

In [17]:
#text_df.columns

In [14]:
# filter for rows that have nan in label and classify these rows
text_df_unlabelled = text_df[text_df['label'].isna()]
text_df_unlabelled.shape

(0, 4)

In [15]:
# Load the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [16]:
unlabelled_texts = [
    "I was unable to work due to a severe headache.",
    "headache all day, busy day at work",
    "pain",
    "pain work",
    "I completed all my tasks without any issues.",
    "Feeling very ill and couldn't concentrate on work.",
    "Good day today",
    "Worked today",
    "Felt ill but stayed at work",
    "surgery",
    "Surgery",
    "sick day"
]

candidate_labels = ['yes', 'no']

# Classify each text
for text in unlabelled_texts:
    hypothesis_template = "The following is a subject’s self-report: {}. Are severe symptoms recorded that affected the subject's ability to work?"
    result = classifier(text, candidate_labels, hypothesis_template=hypothesis_template)
    label = result['labels'][0]  # The label with the highest score
    score = round(result['scores'][0], 2)  # The corresponding score
    print(f"Text: {text}\nLabel: {label}, Score:{score}\n")


Text: I was unable to work due to a severe headache.
Label: yes, Score:0.51

Text: headache all day, busy day at work
Label: yes, Score:0.56

Text: pain
Label: yes, Score:0.63

Text: pain work
Label: yes, Score:0.55

Text: I completed all my tasks without any issues.
Label: no, Score:0.65

Text: Feeling very ill and couldn't concentrate on work.
Label: yes, Score:0.55

Text: Good day today
Label: no, Score:0.65

Text: Worked today
Label: no, Score:0.68

Text: Felt ill but stayed at work
Label: no, Score:0.6

Text: surgery
Label: yes, Score:0.56

Text: Surgery
Label: yes, Score:0.53

Text: sick day
Label: yes, Score:0.6



In [ ]:
# Loop through each text in 'text' column and update 'label'
for index, text in text_df['text'].items():
    hypothesis_template = "The following is a subject’s self-report: {}. Are severe symptoms recorded that affected the subject's ability to work?"
    result = classifier(text, candidate_labels, hypothesis_template=hypothesis_template)
    label = result['labels'][0]  # The label with the highest score
    print(f"{index} Text: {text}\nLabel: {label}\n")
    text_df.loc[index, 'label'] = label


In [ ]:
text_df = text_df.drop(columns=['text'])
text_df.head()

In [11]:
text_df['work_affected'] = text_df['label'].replace({'yes': 1, 'no': 0})
text_df

,ID,userID,label,work_affected
0,276,113,yes,1
1,278,113,yes,1
2,288,113,no,0
3,299,113,yes,1
4,300,92,no,0
...,...,...,...,...
6222,32149,5527,no,0
6223,32152,318,yes,1
6224,32153,7647,no,0
6225,32157,7166,yes,1


In [13]:
# Save data locally
text_df.to_csv('data/text_df.csv', index=False)
#text_df.to_csv('/content/drive/MyDrive/text_df.csv', index=False)

In [3]:
# Read in saved data
file_path = 'data/text_df.csv'
text_df = pd.read_csv(file_path)
text_df.shape

(6227, 4)

In [7]:
text_df['label'].value_counts()

label
yes    3135
no     3092
Name: count, dtype: int64

In [9]:
label_counts = text_df.groupby('userID')['label'].value_counts()
label_counts = label_counts.reset_index()
label_counts.head()

,userID,label,count
0,92,yes,2
1,92,no,1
2,93,yes,4
3,103,no,1
4,103,yes,1


In [11]:
label_counts_pv = label_counts.pivot_table(index='userID', columns='label', values='count', fill_value=0)
label_counts_pv.reset_index(inplace=True)
label_counts_pv.head()

label,userID,no,yes
0,92,1,2
1,93,0,4
2,103,1,1
3,113,48,37
4,117,1,0


In [13]:
# Count the number of users that report symptoms that affected work
yes_freq = label_counts_pv['yes'].value_counts().reset_index()
yes_freq.head()

,yes,count
0,0,379
1,1,249
2,2,149
3,3,54
4,4,43


In [23]:
note_takers = len(label_counts_pv)
print("Total note takers", note_takers)

zero_yes_users = yes_freq.loc[yes_freq['yes'] == 0.0, 'count'].values[0]
print("Users that did not report symptoms that affect work", zero_yes_users)

yes_users = note_takers - zero_yes_users
print("Users that did report symptoms affecting work at least once", yes_users)

print("Percentage of note takers that recorded symptoms that affect  ability to work: ", round(yes_users*100/note_takers, 1), "%")

print("Percentage of total users recorded symptoms that affect  ability to work: ", round(yes_users*100/users_total, 1), "%")


Total note takers 1028
Users that did not report symptoms that affect work 379
Users that did report symptoms affecting work at least once 649
Percentage of note takers that recorded symptoms that affect  ability to work:  63.1 %
Percentage of total users recorded symptoms that affect  ability to work:  33.8 %
